In [ ]:
import numpy as np
import crystalpy as crp
import matplotlib.pyplot as plt
import matplotlib

## 1st dislocation

In [ ]:
l0 = crp.generate.create_lattice(
    dimensions=(5, 5, 3),  #  The number of cells
    cell="B4_ZnS",
)

In [ ]:
crp.visualization.render(molecule=l0, window_size=(800, 800))

In [ ]:
b0 = [1, 0, 0]
position0 = [5, 4.7, 7.5]
plane0 = (0, 1, 0)

u0 = crp.dislocations.displace_love(
    crystal=l0,
    position=position0,
    burgers_vector=b0,
    plane=plane0,
    bv_fraction=1.5,
)

In [ ]:
%matplotlib widget
plt.xlabel("OX ($\AA$)")
plt.ylabel("OZ ($\AA$)")
plt.title("Dislocation field (u), y = 0 projection")
y_dim = 2
x_dim = 0

plt.quiver(l0.coordinates[..., x_dim], l0.coordinates[..., y_dim], u0[..., x_dim], u0[..., y_dim], 
           color=crp.visualization.vectors_to_rgb(u0[..., (x_dim, y_dim)]))
plt.scatter(position0[x_dim], position0[y_dim], s=200.5, c="brown", marker="1")
plt.tight_layout()
plt.gca().invert_yaxis()

In [ ]:
l1 = l0.translate(u0)
crp.visualization.render(molecule=l0+u0, window_size=(800, 800))

In [ ]:
def beta(burgers_vector, bv_fraction, plane, cell, x):
    rt = crp.dislocations._get_rotation_tensor(
        burgers_vector=burgers_vector,
        plane=plane,
        cell=cell
    )
    burgers_vector = bv_fraction*cell.to_cartesian_indices(burgers_vector)
    burgers_vector = burgers_vector.reshape(-1, 1)
    burgers_vector = rt.dot(burgers_vector).squeeze()

    be = np.sqrt(burgers_vector[0]**2 + burgers_vector[1]**2)
    bz = burgers_vector[2]
    return crp.dislocations.beta_function(x, be=be, bz=bz)

In [ ]:
beta0 = beta(burgers_vector=b0, bv_fraction=0.5, plane=plane0, cell=l1.cell, x=l1.coordinates)

## 2nd dislocation

In [ ]:
b1 = [1, 0, 0]
b1_fraction = 0.5
position1 = np.asarray([3.3, 0.0, 1.8]).reshape(1, -1)
plane1 = (0, 1, 0)

$\beta_0 \rightarrow F$

Pytania:
- beta0_b1?

In [ ]:
one = np.diag(np.ones(3)).reshape(1, 3, 3)
beta0_b1 = beta(burgers_vector=b0, bv_fraction=0.5, plane=plane0, cell=l1.cell, x=position1)
F = one-beta0_b1
# TODO czy ponizsze jest poprawne? (w przecwinym razie mamy macierz nieosobliowa)
F = F[:, :2, :2]  # 2x2 matrix
F = np.linalg.inv(F)
result = np.zeros((3, 3))
result[:2, :2] = F.squeeze()
F = result
F

Rotate b1

In [ ]:
b1_actual = F.dot(b1)
b1_actual

$b_1 \rightarrow \beta_1$

In [ ]:
beta1 = beta(burgers_vector=b1, bv_fraction=b1_fraction, plane=plane1, cell=l1.cell, x=l1.coordinates)
beta1[0]